In [1]:
import requests as req
from urllib.parse import urljoin
import bs4
import logging

In [85]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

class Crawler:

    def __init__(self, urls=[], file_counter=0):
        self.visited_urls = {}
        self.urls_to_visit = urls
        self.file_counter = file_counter
        self.start_of_url = 'https://en.wikipedia.org'


    def download_url(self, url):
        return req.get(url).text

    def write_text_to_file(self, article):
        if article:
            self.file_counter += 1
            path = 'DIR/' + str(self.file_counter) + '.txt'

            with open(path, 'w+', encoding='utf-8') as f:
                f.write(article.text)



    def get_linked_a(self,html):
        
        soup = bs4.BeautifulSoup(html, 'html.parser')
        article = soup.find('div', {"class":"mw-parser-output"})
        self.write_text_to_file(article)
        a = {}
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/wiki') and path[6:11] != 'File:':
                a[path] = path
        return a


    def crawl(self, url):
        
        html = self.download_url(url)

        for singleAtag in self.get_linked_a(html).values():

            if singleAtag and (len(self.urls_to_visit < 200)) (singleAtag not in self.urls_to_visit) and(singleAtag not in self.visited_urls):
                self.urls_to_visit.append(singleAtag)





    def run(self):
        limit = 50001
        while self.urls_to_visit and self.file_counter<limit:
            tmp = self.urls_to_visit.pop(0)
            url = str(self.start_of_url) + str(tmp)
            try:
                logging.info(f'Crawling: {url}')
                self.visited_urls[tmp] = 1
                self.crawl(url)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')



In [86]:
if __name__ == '__main__':
    clawler = Crawler(urls=['/wiki/Leonardo_da_Vinci'])
    clawler.run()

2022-05-11 19:51:48,958 INFO:Crawling: https://en.wikipedia.org/wiki/Leonardo_da_Vinci
2022-05-11 19:51:49,641 INFO:Crawling: https://en.wikipedia.org/wiki/Wikipedia:Good_articles
2022-05-11 19:51:49,861 INFO:Crawling: https://en.wikipedia.org/wiki/Wikipedia:Protection_policy#semi
2022-05-11 19:51:50,166 INFO:Crawling: https://en.wikipedia.org/wiki/Da_Vinci_(disambiguation)
2022-05-11 19:51:50,332 INFO:Crawling: https://en.wikipedia.org/wiki/Leonardo_da_Vinci_(disambiguation)
2022-05-11 19:51:50,494 INFO:Crawling: https://en.wikipedia.org/wiki/Republic_of_Florence
2022-05-11 19:51:50,872 INFO:Crawling: https://en.wikipedia.org/wiki/Surname
2022-05-11 19:51:51,271 INFO:Crawling: https://en.wikipedia.org/wiki/Given_name
2022-05-11 19:51:51,600 INFO:Crawling: https://en.wikipedia.org/wiki/Francesco_Melzi
2022-05-11 19:51:51,846 INFO:Crawling: https://en.wikipedia.org/wiki/Anchiano
2022-05-11 19:51:52,020 INFO:Crawling: https://en.wikipedia.org/wiki/Vinci,_Tuscany
2022-05-11 19:51:52,253 I

KeyboardInterrupt: 